In [1]:
import sys
from __future__ import division, print_function
import sys
sys.path.insert(0,'../code')
import book_format
book_format.load_style('../code')

# Computational Statistics

## Distributions

In statistics a <span>**distribution**</span> is a set of values and
their corresponding probabilities.

For example, if you roll a six-sided die, the set of possible values is
the numbers 1 to 6, and the probability associated with each value is
1/6.

As another example, you might be interested in how many times each word
appears in common English usage. You could build a distribution that
includes each word and how many times it appears.

To represent a distribution in Python, you could use a dictionary that
maps from each value to its probability. I have written a class called
<span>Pmf</span> that uses a Python dictionary in exactly that way, and
provides a number of useful methods. I called the class Pmf in reference
to a <span>**probability mass function**</span>, which is a way to
represent a distribution mathematically.

<span>Pmf</span> is defined in a Python module I wrote to accompany this
book, <span>thinkbayes.py</span>. You can download it from
<http://thinkbayes.com/thinkbayes.py>. For more information see
Section [download].

To use <span>Pmf</span> you can import it like this:

In [2]:
from thinkbayes import Pmf

The following code builds a Pmf to represent the distribution of
outcomes for a six-sided die:

In [3]:
pmf = Pmf()
for x in [1,2,3,4,5,6]:
    pmf.Set(x, 1/6.0)
pmf.Print()

1 0.16666666666666666
2 0.16666666666666666
3 0.16666666666666666
4 0.16666666666666666
5 0.16666666666666666
6 0.16666666666666666


`Pmf` creates an empty Pmf with no values. The `Set` method sets the
probability associated with each value to $1/6$.

Here’s another example that counts the number of times each word appears
in a sequence:

In [4]:
word_list = ['hi', 'the', 'bye', 'hi', 'football', 'sky']
pmf = Pmf()
for word in word_list:
    pmf.Incr(word, 1)
pmf.Print()

bye 1
football 1
hi 2
sky 1
the 1


`Incr` increases the “probability” associated with each word by 1. If a
word is not already in the Pmf, it is added.

I put “probability” in quotes because in this example, the probabilities
are not normalized; that is, they do not add up to 1. So they are not
true probabilities.

But in this example the word counts are proportional to the
probabilities. So after we count all the words, we can compute
probabilities by dividing through by the total number of words.
<span>Pmf</span> provides a method, `Normalize`, that does exactly that:

In [5]:
pmf.Normalize()
pmf.Print()

bye 0.16666666666666666
football 0.16666666666666666
hi 0.3333333333333333
sky 0.16666666666666666
the 0.16666666666666666


Once you have a Pmf object, you can ask for the probability associated
with any value:

In [6]:
print(pmf.Prob('the'))

0.16666666666666666


And that would print the frequency of the word “the” as a fraction of
the words in the list.

Pmf uses a Python dictionary to store the values and their
probabilities, so the values in the Pmf can be any hashable type. The
probabilities can be any numerical type, but they are usually
floating-point numbers (type `float`).

## The cookie problem

In the context of Bayes’s theorem, it is natural to use a Pmf to map
from each hypothesis to its probability. In the cookie problem, the
hypotheses are $B_1$ and $B_2$. In Python, I represent them with
strings:

In [7]:
pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

This distribution, which contains the priors for each hypothesis, is
called (wait for it) the <span>**prior distribution**</span>.

To update the distribution based on new data (the vanilla cookie), we
multiply each prior by the corresponding likelihood. The likelihood of
drawing a vanilla cookie from Bowl 1 is 3/4. The likelihood for Bowl 2
is 1/2.

In [8]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

`Mult` does what you would expect. It gets the probability for the given
hypothesis and multiplies by the given likelihood.

After this update, the distribution is no longer normalized, but because
these hypotheses are mutually exclusive and collectively exhaustive, we
can <span>**renormalize**</span>:

In [9]:
pmf.Normalize()

0.625

The result is a distribution that contains the posterior probability for
each hypothesis, which is called (wait now) the <span>**posterior
distribution**</span>.

Finally, we can get the posterior probability for Bowl 1:

In [10]:
print(pmf.Prob('Bowl 1'))

0.6000000000000001


And the answer is 0.6. You can download this example from
<http://thinkbayes.com/cookie.py>. For more information see
Section [download].

## The Bayesian framework

Before we go on to other problems, I want to rewrite the code from the
previous section to make it more general. First I’ll define a class to
encapsulate the code related to this problem:

In [11]:
class Cookie(Pmf):
    """A map from string bowl ID to probablity."""

    def __init__(self, hypos):
        """Initialize self.

        hypos: sequence of string bowl IDs
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates the PMF with new data.

        data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
        }

    def Likelihood(self, data, hypo):
        """The likelihood of the data under the hypothesis.

        data: string cookie type
        hypo: string bowl ID
        """
        mix = self.mixes[hypo]
        like = mix[data]
        return like

A Cookie object is a Pmf that maps from hypotheses to their
probabilities. The `__init__` method gives each hypothesis the same
prior probability. As in the previous section, there are two hypotheses:

In [12]:
hypos = ['Bowl 1', 'Bowl 2']
pmf = Cookie(hypos)

`Cookie` provides an `Update` method that takes data as a parameter and
updates the probabilities. `Update` loops through each hypothesis in the suite and multiplies its
probability by the likelihood of the data under the hypothesis, which is
computed by `Likelihood`.

`Likelihood` uses `mixes`, which is a dictionary that maps from the name
of a bowl to the mix of cookies in the bowl.

Here’s what the update looks like:

In [13]:
pmf.Update('vanilla')

And then we can print the posterior probability of each hypothesis:

In [14]:
for hypo, prob in pmf.Items():
    print(hypo, prob)

Bowl 1 0.6000000000000001
Bowl 2 0.4


which is the same as what we got before. This code is more complicated
than what we saw in the previous section. One advantage is that it
generalizes to the case where we draw more than one cookie from the same
bowl (with replacement):

In [15]:
dataset = ['vanilla', 'chocolate', 'vanilla']
for data in dataset:
    pmf.Update(data)
pmf.Print()

Bowl 1 0.627906976744186
Bowl 2 0.37209302325581395


The other advantage is that it provides a framework for solving many
similar problems. In the next section we’ll solve the Monty Hall problem
computationally and then see what parts of the framework are the same.

The code in this section is available from
<http://thinkbayes.com/cookie2.py>. For more information see
Section [download].

## The Monty Hall problem

To solve the Monty Hall problem, I’ll define a new class:

In [16]:
class Monty(Pmf):
    """Map from string location of car to probability"""

    def __init__(self, hypos):
        """Initialize the distribution.

        hypos: sequence of hypotheses
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates each hypothesis based on the data.

        data: any representation of the data
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

    def Likelihood(self, data, hypo):
        """Compute the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

So far `Monty` and `Cookie` are nearly the same. And the code that
creates the Pmf is the same, too, except for the names of the
hypotheses:

In [17]:
hypos = 'ABC'
pmf = Monty(hypos)

Calling `Update` is pretty much the same:

In [18]:
data = 'B'
pmf.Update(data)

And the implementation of `Update` is exactly the same. The only part that requires some work is `Likelihood`:

```python
def Likelihood(self, data, hypo):
    if hypo == data:
        return 0
    elif hypo == 'A':
        return 0.5
    else:
        return 1
```
Finally, printing the results is the same:

In [19]:
for hypo, prob in pmf.Items():
    print(hypo, prob)

B 0.0
A 0.3333333333333333
C 0.6666666666666666


In this example, writing `Likelihood` is a little complicated, but the
framework of the Bayesian update is simple. The code in this section is
available from <http://thinkbayes.com/monty.py>. For more information
see Section [download].

## Encapsulating the framework

Now that we see what elements of the framework are the same, we can
encapsulate them in an object—a `Suite` is a `Pmf` that provides
`__init__`, `Update`, and `Print`:

```python
class Suite(Pmf):
        """Represents a suite of hypotheses and their probabilities."""

        def __init__(self, hypo=tuple()):
            """Initializes the distribution."""

        def Update(self, data):
            """Updates each hypothesis based on the data."""

        def Print(self):
            """Prints the hypotheses and their probabilities."""
```

The implementation of `Suite` is in `thinkbayes.py`. To use `Suite`, you
should write a class that inherits from it and provides `Likelihood`.
For example, here is the solution to the Monty Hall problem rewritten to
use `Suite`:

In [20]:
from thinkbayes import Suite

class Monty(Suite):

    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

And here’s the code that uses this class:

In [21]:
suite = Monty('ABC')
suite.Update('B')
suite.Print()

A 0.3333333333333333
B 0.0
C 0.6666666666666666


You can download this example from <http://thinkbayes.com/monty2.py>.
For more information see Section [download].

## The <span>M&M</span> problem

We can use the `Suite` framework to solve the <span>M&M</span> problem.
Writing the `Likelihood` function is tricky, but everything else is
straightforward.

In [22]:
class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""

    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

First I need to encode the color mixes from before and after 1995:

```python
mix94 = dict(brown=30,
             yellow=20,
             ...

mix96 = dict(blue=24,
             green=20,
             ...
```

Then I have to encode the hypotheses:

```python
hypoA = dict(bag1=mix94, bag2=mix96)
hypoB = dict(bag1=mix96, bag2=mix94)
```

`hypoA` represents the hypothesis that Bag 1 is from 1994 and Bag 2 from 1996. `hypoB` is the other way around.

Next I map from the name of the hypothesis to the representation:

```python
hypotheses = dict(A=hypoA, B=hypoB)
```

And finally I can write `Likelihood`. In this case the hypothesis,
`hypo`, is a string, either `A` or `B`. The data is a tuple that
specifies a bag and a color.

        def Likelihood(self, data, hypo):
            bag, color = data
            mix = self.hypotheses[hypo][bag]
            like = mix[color]
            return like

Here’s the code that creates the suite and updates it:

In [23]:
suite = M_and_M('AB')

suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))

suite.Print()

A 0.7407407407407407
B 0.2592592592592592


The posterior probability of A is approximately $20/27$, which is what
we got before.

The code in this section is available from
<http://thinkbayes.com/m_and_m.py>. For more information see
Section [download].

## Discussion

This chapter presents the Suite class, which encapsulates the Bayesian
update framework.

<span>Suite</span> is an <span>**abstract type**</span>, which means
that it defines the interface a Suite is supposed to have, but does not
provide a complete implementation. The <span>Suite</span> interface
includes <span>Update</span> and <span>Likelihood</span>, but the
<span>Suite</span> class only provides an implementation of
<span>Update</span>, not <span>Likelihood</span>.

A <span>**concrete type**</span> is a class that extends an abstract
parent class and provides an implementation of the missing methods. For
example, <span>Monty</span> extends <span>Suite</span>, so it inherits
<span>Update</span> and provides <span>Likelihood</span>.

If you are familiar with design patterns, you might recognize this as an
example of the template method pattern. You can read about this pattern
at <http://en.wikipedia.org/wiki/Template_method_pattern>.

Most of the examples in the following chapters follow the same pattern;
for each problem we define a new class that extends <span>Suite</span>,
inherits <span>Update</span>, and provides <span>Likelihood</span>. In a
few cases we override <span>Update</span>, usually to improve
performance.

## Exercises

In Section [framework] I said that the solution to the cookie problem
generalizes to the case where we draw multiple cookies with replacement.

But in the more likely scenario where we eat the cookies we draw, the
likelihood of each draw depends on the previous draws.

Modify the solution in this chapter to handle selection without
replacement. Hint: add instance variables to <span>Cookie</span> to
represent the hypothetical state of the bowls, and modify
<span>Likelihood</span> accordingly. You might want to define a
<span>Bowl</span> object.